# General imports

In [8]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

#Delete all folders in data

In [25]:
import os
for root, dirs, files in os.walk("Data", topdown=True):
    for name in files:
        os.remove(os.path.join(root, name))
    for name in dirs:
        os.rmdir(os.path.join(root, name))

os.rmdir("Data")

FileNotFoundError: ignored

#Install special libs

In [9]:
!pip install pydub
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#DataEngine

In [19]:
import wget
import json
import glob
from pydub import AudioSegment
import librosa
import os
import shutil
import matplotlib.pyplot as plt
import librosa.display


def downloas_bird_sounds(query):
    """
        Downloads the bird sounds from www.xeno-canto.org
        param:
          query: Name of the bird. Blank space with %20
    """
    # GET ALL FILENAMES AND PATHS OF THE QUERY BIRD
    url = 'https://www.xeno-canto.org/api/2/recordings?query=' + query

    dataFolder = "Data/" + query.replace("%20", "_") + "/"
    mp3Folder = dataFolder + "mp3/"
    arrayFolder = dataFolder + "arrays/"
    predTestFolder = dataFolder + "preTest/"

    os.mkdir("Data")
    os.mkdir(dataFolder)
    os.mkdir(mp3Folder)
    os.mkdir(arrayFolder)
    os.mkdir(predTestFolder)

    filename = wget.download(url, dataFolder + 'recordings.json')
    print(filename)

    # Get the json entries from your downloaded json
    jsonFile = open(dataFolder + 'recordings.json', 'r')
    values = json.load(jsonFile)
    jsonFile.close()

    # Create a pandas dataframe of records & convert to .csv file
    record_df = pd.DataFrame(values['recordings'])
    record_df.to_csv(dataFolder + 'xc-noca.csv', index=False)

    # Make wget input file
    url_list = []
    for file in record_df['file'].tolist():
        url_list.append('{}'.format(file))
    with open(dataFolder + 'xc-noca-urls.txt', 'w+') as f:
        for item in url_list:
            f.write("{}\n".format(item))

    # Get all soundfiles
    os.system('wget -P ' + mp3Folder + ' --trust-server-names -i' + dataFolder + 'xc-noca-urls.txt')

    files = os.listdir(mp3Folder)
    [os.replace(mp3Folder + file, mp3Folder + file.replace(" ", "_")) for file in files]


def prepare_dataset(query, nbrOfTestSoundsForPrediction):
    """
        Prepares the dataset for using in model
        :param
        query: Name of the bird. Blank space with %20
        nbrOfTestSoundsForPrediction: Number of sounds copied to a
        seperate folder for later predictions
    """
    dataFolder = "Data/" + query.replace("%20", "_") + "/"
    mp3Folder = dataFolder + "mp3/"
    arrayFolder = dataFolder + "arrays/"
    predTestFolder = dataFolder + "preTest/"

    os.mkdir("tmp")

    # The following line is only needed once if ffmpeg is not part of the PATH variables
    # os.environ["PATH"] += os.pathsep + r'F:\ffmpeg\bin'

    # Reformat path string
    globlist = glob.glob(mp3Folder + "*.mp3")
    new_list = []
    for string in globlist:
        new_string = string.replace("\\", "/")
        new_list.append(new_string)

    # Copy 5 entries to /predTest
    last_elements = new_list[-nbrOfTestSoundsForPrediction:]
    print(last_elements)

    for file in last_elements:
        shutil.copy(file, predTestFolder)
        os.remove(file)

    globlist.clear()
    globlist = glob.glob(mp3Folder + "*.mp3")
    new_list.clear()
    for string in globlist:
        new_string = string.replace("\\", "/")
        new_list.append(new_string)

    # Extract frequencies and save them as np array
    for file in new_list:
        src = file
        dst = "tmp/tmp.wav"

        # convert mp3 to wav
        sound = AudioSegment.from_mp3(src)
        ten_seconds = 10 * 1000
        first_10_seconds = sound[:ten_seconds]
        first_10_seconds.export(dst, format="wav")

        y, sr = librosa.load(dst)

        # CREATE A FIXED LENGTH
        librosa.util.fix_length(y, 220500)

        # EXTRACT FEATURES
        mfccs_features = librosa.feature.mfcc(y, sr, n_mfcc=40)
        mfccs_scaled_features = np.mean(mfccs_features.T, axis=0)

        # SAVE FEATURES TO FILE
        index = new_list.index(file)
        arrayPath = arrayFolder + str(index)
        np.save(arrayPath, mfccs_scaled_features)

        # Remove temp wav file
        os.remove(dst)

    # CREATE AN ARRAY OF ALL PICTURE FEATURES AND SAVE IT
    arraylist = glob.glob(arrayFolder + "*.npy")
    extracted_features = []
    for file in arraylist:
        data = np.load(file)
        label = query.replace("%20", "_")
        extracted_features.append([data, label])

    np.save(arrayFolder + "summery_array", extracted_features)


def download_and_prepare_bird_dataset(query, nbrOfTestSoundsForPrediction):
    """
        Downloads & prepares the dataset for using in model
        :param
        query: Name of the bird. Blank space with %20
        nbrOfTestSoundsForPrediction: Number of sounds copied to a
        seperate folder for later predictions
    """
    downloas_bird_sounds(query)
    prepare_dataset(query, nbrOfTestSoundsForPrediction)


def show_spectogram_for_mp3(filepath):
    """
        Shows the spectogram of a given mp3 path
        :param
        filepath: The filepath of the mp3
    """
    src = filepath
    dst = "tmp/tmp.wav"

    # convert wav to mp3
    sound = AudioSegment.from_mp3(src)
    ten_seconds = 10 * 1000
    first_10_seconds = sound[:ten_seconds]
    first_10_seconds.export(dst, format="wav")

    y, sr = librosa.load(dst)
    librosa.util.fix_length(y, 220500)

    # SHOW WAVE
    fig, ax = plt.subplots(nrows=3, sharex=True)
    librosa.display.waveshow(y, sr=sr, ax=ax[0])
    ax[0].set(title='Wave')

    # SHOW SPEC
    D = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)
    img = librosa.display.specshow(D, y_axis='linear', x_axis='time', sr=sr, ax=ax[1])
    ax[1].set(title='Linear-frequency power spectrogram')

    # SHOW EXTRACTED FEATURES
    mfccs_features = librosa.feature.mfcc(y, sr, n_mfcc=40)
    img = librosa.display.specshow(mfccs_features, x_axis='time', ax=ax[2])
    ax[2].set(title='mfccs_features')
    plt.show()




def perpare_mp3_for_prediction(filepath):
    """
        Prepares the given mp3 for prediction
        :param
        filepath: The filepath of the mp3
    """
    src = filepath
    dst = "tmp/tmp2.wav"

    # convert mp3 to wav
    sound = AudioSegment.from_mp3(src)
    ten_seconds = 10 * 1000
    first_10_seconds = sound[:ten_seconds]
    first_10_seconds.export(dst, format="wav")

    y, sr = librosa.load(dst)

    # FIXED LENGTH
    librosa.util.fix_length(y, 220500)

    # EXTRACT FEATURES
    mfccs_features = librosa.feature.mfcc(y, sr, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T, axis=0)

    # RESHAPE
    mfccs_scaled_features = mfccs_scaled_features.reshape(1, -1)

    # Remove temp wav file
    os.remove(dst)

    return mfccs_scaled_features


def get_dataframe(query):
    dataFolder = "Data/" + query.replace("%20", "_") + "/"
    arrayFolder = dataFolder + "arrays/"

    numpy_data = np.load(arrayFolder + "summery_array.npy", allow_pickle=True)
    extracted_features_df = pd.DataFrame(numpy_data, columns=['feature', 'label'])

    return extracted_features_df


def get_concat_dataframe(query_list):
    result = pd.DataFrame()

    for query in query_list:
        df = get_dataframe(query)
        result = result.append(df)

    return result

#Models

In [11]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D


def create_and_fit_dense_model(batch_size, epochs, callbacks, X_train, X_test, y_train, y_test):
    # BUILD THE MODEL
    model = Sequential()
    model.add(Dense(100, input_shape=(40,)))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(200))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(100))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

    # TRAIN THE MODEL
    history = model.fit(X_train, y_train, batch_size=batch_size, callbacks=callbacks, epochs=epochs,
                        validation_data=(X_test, y_test))

    # SAVE MODEL AND HISTORY_DATA
    np.save('Auswertung/history.npy', history.history)
    model.save('Auswertung/model')

    return history


def create_and_fit_cnn_model(batch_size, epochs, callbacks, X_train, X_test, y_train, y_test):
    # BUILD THE MODEL
    model = Sequential()
    model.add(Conv1D(64, 3, activation='relu', input_shape=(40, 1)))
    model.add(Conv1D(64, 3, activation='relu'))
    model.add(MaxPooling1D(3))
    model.add(Conv1D(128, 3, activation='relu'))
    model.add(Conv1D(128, 3, activation='relu'))
    model.add(GlobalAveragePooling1D())
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

    # TRAIN THE MODEL
    history_CNN = model.fit(X_train, y_train, batch_size=batch_size, callbacks=callbacks, epochs=epochs,
                            validation_data=(X_test, y_test))

    # SAVE MODEL AND HISTORY_DATA
    np.save('Auswertung/history_CNN.npy', history_CNN.history)
    model.save('Auswertung/model_CNN')

    return history_CNN

#General helper functions

In [12]:
import tensorflow as tf


# Create a function to import an image and resize it to be able to be used with our model
def load_and_prep_image(filename, img_shape=224, scale=True):
    """
    Reads in an image from filename, turns it into a tensor and reshapes into
    (224, 224, 3).
    Parameters
    ----------
    filename (str): string filename of target image
    img_shape (int): size to resize target image to, default 224
    scale (bool): whether to scale pixel values to range(0, 1), default True
    """
    # Read in the image
    img = tf.io.read_file(filename)
    # Decode it into a tensor
    img = tf.image.decode_jpeg(img)
    # Resize the image
    img = tf.image.resize(img, [img_shape, img_shape])
    if scale:
        # Rescale the image (get all values between 0 and 1)
        return img / 255.
    else:
        return img


# Note: The following confusion matrix code is a remix of Scikit-Learn's
# plot_confusion_matrix function - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.plot_confusion_matrix.html
import itertools
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix


# Our function needs a different name to sklearn's plot_confusion_matrix
def make_confusion_matrix(y_true, y_pred, classes=None, figsize=(10, 10), text_size=15, norm=False, savefig=False):
    """Makes a labelled confusion matrix comparing predictions and ground truth labels.
    If classes is passed, confusion matrix will be labelled, if not, integer class values
    will be used.
    Args:
      y_true: Array of truth labels (must be same shape as y_pred).
      y_pred: Array of predicted labels (must be same shape as y_true).
      classes: Array of class labels (e.g. string form). If `None`, integer labels are used.
      figsize: Size of output figure (default=(10, 10)).
      text_size: Size of output figure text (default=15).
      norm: normalize values or not (default=False).
      savefig: save confusion matrix to file (default=False).
    Returns:
      A labelled confusion matrix plot comparing y_true and y_pred.
    Example usage:
      make_confusion_matrix(y_true=test_labels, # ground truth test labels
                            y_pred=y_preds, # predicted labels
                            classes=class_names, # array of class label names
                            figsize=(15, 15),
                            text_size=10)
    """
    # Create the confustion matrix
    cm = confusion_matrix(y_true, y_pred)
    cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]  # normalize it
    n_classes = cm.shape[0]  # find the number of classes we're dealing with

    # Plot the figure and make it pretty
    fig, ax = plt.subplots(figsize=figsize)
    cax = ax.matshow(cm, cmap=plt.cm.Blues)  # colors will represent how 'correct' a class is, darker == better
    fig.colorbar(cax)

    # Are there a list of classes?
    if classes:
        labels = classes
    else:
        labels = np.arange(cm.shape[0])

    # Label the axes
    ax.set(title="Confusion Matrix",
           xlabel="Predicted label",
           ylabel="True label",
           xticks=np.arange(n_classes),  # create enough axis slots for each class
           yticks=np.arange(n_classes),
           xticklabels=labels,  # axes will labeled with class names (if they exist) or ints
           yticklabels=labels)

    # Make x-axis labels appear on bottom
    ax.xaxis.set_label_position("bottom")
    ax.xaxis.tick_bottom()

    # Set the threshold for different colors
    threshold = (cm.max() + cm.min()) / 2.

    # Plot the text on each cell
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if norm:
            plt.text(j, i, f"{cm[i, j]} ({cm_norm[i, j] * 100:.1f}%)",
                     horizontalalignment="center",
                     color="white" if cm[i, j] > threshold else "black",
                     size=text_size)
        else:
            plt.text(j, i, f"{cm[i, j]}",
                     horizontalalignment="center",
                     color="white" if cm[i, j] > threshold else "black",
                     size=text_size)

    # Save the figure to the current working directory
    if savefig:
        fig.savefig("confusion_matrix.png")


# Make a function to predict on images and plot them (works with multi-class)
def pred_and_plot(model, filename, class_names, scale=True, shape=224):
    """
    Imports an image located at filename, makes a prediction on it with
    a trained model and plots the image with the predicted class as the title.
    """
    # Import the target image and preprocess it
    img = load_and_prep_image(filename, shape, scale)

    # Make a prediction
    pred = model.predict(tf.expand_dims(img, axis=0))

    # Get the predicted class
    if len(pred[0]) > 1:  # check for multi-class
        pred_class = class_names[pred.argmax()]  # if more than one output, take the max
    else:
        pred_class = class_names[int(tf.round(pred)[0][0])]  # if only one output, round

    # Plot the image and predicted class
    plt.imshow(img)
    plt.title(f"Prediction: {pred_class}")
    plt.axis(False);
    plt.show()


import datetime


def create_tensorboard_callback(dir_name, experiment_name):
    """
    Creates a TensorBoard callback instand to store log files.
    Stores log files with the filepath:
      "dir_name/experiment_name/current_datetime/"
    Args:
      dir_name: target directory to store TensorBoard log files
      experiment_name: name of experiment directory (e.g. efficientnet_model_1)
    """
    log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir
    )
    print(f"Saving TensorBoard log files to: {log_dir}")
    return tensorboard_callback


# Plot the validation and training data separately
import matplotlib.pyplot as plt


def plot_loss_curves(history):
    """
    Returns separate loss curves for training and validation metrics.
    Args:
      history: TensorFlow model History object (see: https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/History)
    """
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']

    epochs = range(len(history.history['loss']))

    # Plot loss
    plt.plot(epochs, loss, label='training_loss')
    plt.plot(epochs, val_loss, label='val_loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.legend()
    plt.show()

    # Plot accuracy
    plt.figure()
    plt.plot(epochs, accuracy, label='training_accuracy')
    plt.plot(epochs, val_accuracy, label='val_accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.legend();
    plt.show()


def launchTensorBoard(log_dir):
    import os
    os.system('tensorboard --logdir=' + log_dir)
    return


def compare_historys(original_history, new_history, initial_epochs=5):
    """
    Compares two TensorFlow model History objects.
    Args:
      original_history: History object from original model (before new_history)
      new_history: History object from continued model training (after original_history)
      initial_epochs: Number of epochs in original_history (new_history plot starts from here)
    """

    # Get original history measurements
    acc = original_history.history["accuracy"]
    loss = original_history.history["loss"]

    val_acc = original_history.history["val_accuracy"]
    val_loss = original_history.history["val_loss"]

    # Combine original history with new history
    total_acc = acc + new_history.history["accuracy"]
    total_loss = loss + new_history.history["loss"]

    total_val_acc = val_acc + new_history.history["val_accuracy"]
    total_val_loss = val_loss + new_history.history["val_loss"]

    # Make plots
    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(total_acc, label='Training Accuracy')
    plt.plot(total_val_acc, label='Validation Accuracy')
    plt.plot([initial_epochs - 1, initial_epochs - 1],
             plt.ylim(), label='Start Fine Tuning')  # reshift plot around epochs
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(2, 1, 2)
    plt.plot(total_loss, label='Training Loss')
    plt.plot(total_val_loss, label='Validation Loss')
    plt.plot([initial_epochs - 1, initial_epochs - 1],
             plt.ylim(), label='Start Fine Tuning')  # reshift plot around epochs
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')
    plt.show()


# Create function to unzip a zipfile into current working directory
# (since we're going to be downloading and unzipping a few files)
import zipfile


def unzip_data(filename):
    """
    Unzips filename into the current working directory.
    Args:
      filename (str): a filepath to a target zip folder to be unzipped.
    """
    zip_ref = zipfile.ZipFile(filename, "r")
    zip_ref.extractall()
    zip_ref.close()


# Walk through an image classification directory and find out how many files (images)
# are in each subdirectory.
import os


def walk_through_dir(dir_path):
    """
    Walks through dir_path returning its contents.
    Args:
      dir_path (str): target directory
    Returns:
      A print out of:
        number of subdiretories in dir_path
        number of images (files) in each subdirectory
        name of each subdirectory
    """
    for dirpath, dirnames, filenames in os.walk(dir_path):
        print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")


# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


def calculate_results(y_true, y_pred):
    """
    Calculates model accuracy, precision, recall and f1 score of a binary classification model.
    Args:
        y_true: true labels in the form of a 1D array
        y_pred: predicted labels in the form of a 1D array
    Returns a dictionary of accuracy, precision, recall, f1-score.
    """
    # Calculate model accuracy
    model_accuracy = accuracy_score(y_true, y_pred) * 100
    # Calculate model precision, recall and f1 score using "weighted average
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
    model_results = {"accuracy": model_accuracy,
                     "precision": model_precision,
                     "recall": model_recall,
                     "f1": model_f1}
    return model_results


# View an image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random


def view_random_image(target_dir, target_class):
    # Setup target directory (we'll view images from here)
    target_folder = target_dir + target_class

    # Get a random image path
    random_image = random.sample(os.listdir(target_folder), 1)

    # Read in the image and plot it using matplotlib
    img = mpimg.imread(target_folder + "/" + random_image[0])
    plt.imshow(img)
    plt.title(target_class)
    plt.axis("off");
    plt.show();

    print(f"Image shape: {img.shape}")  # show the shape of the image

    return img

def plot_saved_loss_curves(filename):
    """
    Returns separate loss curves for training and validation metrics.
    Args:
      history: TensorFlow model History object (see: https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/History)
    """
    history = np.load(filename, allow_pickle='TRUE').item()

    loss = history['loss']
    val_loss = history['val_loss']

    accuracy = history['sparse_categorical_accuracy']
    val_accuracy = history['val_sparse_categorical_accuracy']

    epochs = range(len(history['loss']))

    # Plot loss
    plt.plot(epochs, loss, label='training_loss')
    plt.plot(epochs, val_loss, label='val_loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.legend()
    plt.show()

    # Plot accuracy
    plt.figure()
    plt.plot(epochs, accuracy, label='training_accuracy')
    plt.plot(epochs, val_accuracy, label='val_accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.legend();
    plt.show()

# Train the model

### DOWNLOAD DATASETS
IS ONLY NEEDED IF THERE'RE NO DATA PREVIOUSLY DOWNLOADED

In [20]:
download_and_prepare_bird_dataset('northern%20cardinal', 5)
download_and_prepare_bird_dataset('Gaviidae', 5)
download_and_prepare_bird_dataset('Crypturellus%20cinereus', 5)

FileExistsError: ignored

###Show Spectograms

In [ ]:
show_spectogram_for_mp3('Data/Gaviidae/preTest/XC681331-Great_Northern_Diver_Skaw_Whalsay_SG_Brambling.mp3')

###Prepare data and callbacks

In [ ]:
# GET DATAFRAMES
train_df = get_concat_dataframe(['northern%20cardinal', 'Gaviidae', 'Crypturellus%20cinereus'])

data = np.array(train_df['feature'].tolist())
labels = np.array(train_df['label'].tolist())

# ENCODE LABELS
labels_encoded = pd.get_dummies(labels)
encoder = LabelEncoder()
encoder.fit(labels)
np.save('Auswertung/classes.npy', encoder.classes_)

# SPLIT TRAINING AND VALIDATION DATA
X_train, X_test, y_train, y_test = train_test_split(data, labels_encoded, shuffle=True, test_size=0.3, random_state=42)

# CALLBACKS
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', min_lr=0.001, patience=10, mode='min', verbose=1)
early_stopping = EarlyStopping(patience=15, monitor='val_accuracy')
tensorboard = create_tensorboard_callback('Auswertung/', 'BirdSoundPrediction')
callbacks = [tensorboard]

###Create and fit model

In [ ]:
#history = create_and_fit_dense_model(32, 148, callbacks, X_train, X_test, y_train, y_test)
history = create_and_fit_cnn_model(32, 15, callbacks, X_train, X_test, y_train, y_test)

# PLOT THE TRAINING CURVES
plot_loss_curves(history)

#Predictions

In [ ]:
from tensorflow import keras

# Load model
#model = keras.models.load_model('Auswertung/model')
model_CNN = keras.models.load_model('Auswertung/model_CNN')

#data = perpare_mp3_for_prediction('Data/Gaviidae/preTest/XC680885-GAVSTE2021-07-14-0644-TOLK-c-m.mp3')
#data = perpare_mp3_for_prediction('Data/northern_cardinal/mp3/CardinalKeyWest.mp3')
#data = perpare_mp3_for_prediction('Data/Crypturellus_cinereus/mp3/Cinereous_tinamou1.mp3')

# Make a prediction
predArray = model_CNN.predict(data)
pred = np.argmax(predArray, axis=1)

print(pred)

classes = np.load('Auswertung/classes.npy', allow_pickle=True)
print(classes[pred])

#Show in Tensorboard

In [ ]:
import threading

t = threading.Thread(target=launchTensorBoard('Auswertung/BirdSoundPrediction/20211223-093239'), args=([]))
t.start()